In [2]:
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab190912.sh

Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 91, done.
remote: Counting objects: 100% (91/91), done.
remote: Compressing objects: 100% (85/85), done.
remote: Total 91 (delta 43), reused 22 (delta 6), pack-reused 0
Unpacking objects: 100% (91/91), done.
/content/Mecab-ko-for-Google-Colab
Installing konlpy.....
     |████████████████████████████████| 19.4 MB 1.3 MB/s 
     |████████████████████████████████| 86 kB 4.8 MB/s 
     |████████████████████████████████| 448 kB 49.8 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2021-11-16 10:57:00--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 104.192.141.1, 2406:da00:ff00

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import urllib.request
from collections import Counter
from konlpy.tag import Mecab
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [10]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/naver_movie_reviews.csv')
df['label'] = np.select([df.score <= 5, df.score > 5], [0, 1])

mecab = Mecab()
stopwords = ['니', '냐', '너무', '좋', '이나', '캣', '입니다', '인데', '그리고', '님', '나', '남', '서', '맘', '많', '할', '임', '그냥', '어요', '봤', '지만', '더', '인', '아니', '않', '기', '싶', '되', '던', '했', '점', '수', '의', '수', '진짜', '본', '알', '보다', '에게', '였', '았','의', '가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다', '고', '하', '을', '다', '영화', '게', '지', '보', '주', '있', '음', '마블', '로', '고양이', '적', '것', '아', '는데', '없', '만', '안', '거', '면', '말', '에서', '었', '습니다', '어', '네요', '듯', '라고', '라', '그', '겠']


def tokenize(data):
  data['tokenized'] = data['comment'].apply(mecab.morphs)
  data['tokenized'] = data['tokenized'].apply(lambda x: [item for item in x if item not in stopwords])


def pre_processing(data):
    _data = data.copy()
    _data.drop_duplicates(subset=['comment'], inplace=True)  # 중복 제거
    _data['comment'] = _data['comment'].str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]', '')  # 한글만
    _data['comment'] = _data['comment'].str.replace('^ +', '')
    _data['comment'].replace('', np.nan, inplace=True)
    _data.dropna(subset=['comment'], inplace=True)

    tokenize(_data)
    
    return _data


train_data, test_data = map(pre_processing, train_test_split(df, test_size=0.25, random_state=42))
negative_words = np.hstack(train_data[train_data.label == 0]['tokenized'].values)
positive_words = np.hstack(train_data[train_data.label == 1]['tokenized'].values)

negative_word_count = Counter(negative_words)
positive_word_count = Counter(positive_words)

X_train = train_data['tokenized'].values
y_train = train_data['label'].values
X_test= test_data['tokenized'].values
y_test = test_data['label'].values

tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

threshold = 3
total_cnt = len(tokenizer.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

vocab_size = total_cnt - rare_cnt + 2
tokenizer = Tokenizer(vocab_size, oov_token = 'OOV') 
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

Counter({'스토리': 1289, '지루': 1101, '같': 1065, '시간': 1053, '네': 872, '재미없': 813, '왜': 796, '노': 768, '성': 767, '잼': 766, '기대': 758, '감독': 750, '함': 741, '편': 736, '내': 718, '돈': 707, '최악': 690, '연기': 678, '느낌': 631, '내용': 624, '다가': 609, '액션': 600, '재미': 584, '평점': 582, '개': 578, '볼': 574, '생각': 570, '마동석': 565, '사람': 540, '정말': 530, '중': 530, '개연': 515, '뭐': 508, '만들': 505, '해서': 501, '분': 500, '까지': 498, '일본': 497, '해': 489, '재밌': 487, '나오': 474, '못': 472, '처음': 471, '배우': 460, '정도': 457, '별로': 457, 'ㅋㅋ': 452, '캐릭터': 422, '건': 421, '모르': 418, '이런': 417, '이렇게': 415, '끝': 412, '실망': 411, '면서': 407, '놈': 403, '줄': 399, '중간': 390, '아깝': 386, '어서': 378, '이게': 375, '장면': 372, '하나': 357, '으면': 354, '때': 352, '전개': 349, '베': 340, '다는': 337, '연출': 336, '다고': 332, '길': 331, '많이': 326, '는지': 324, '이해': 317, '김무열': 312, '원작': 312, '죽': 308, '라는': 308, '걸': 294, '마지막': 287, '한국': 276, '년': 272, '쿠키': 271, '이건': 265, '졸': 263, '감동': 262, '인지': 262, '야': 262, '시': 262, '유치': 260, '시리즈': 258, '부터': 25

Mounted at /content/drive
